In [145]:
from lyricsgenius import Genius
from bs4 import BeautifulSoup
import requests
import pandas as pd
from itertools import islice
import os
import re

# Get top 50 rap songs between 2018 and 2022 from billboard website

In [146]:
def get_top50_from_url(year):
    #example = url_start + '2018' + url_end
    url_start = 'https://www.billboard.com/charts/year-end/'
    url_end = '/hot-rap-songs/'

    url = url_start + str(year) + url_end
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('div', {'class': 'o-chart-results-list-row-container'})

    data = []

    for i in range(len(results)):
        result = results[i]
        title = result.find('h3').text.strip()
        spans = result.find_all('span')
        artist = spans[1].text.strip()
        data.append({'position': i+1, 'title': title, 'artist': artist, 'year': year})

    df = pd.DataFrame(data)
    return df


In [147]:
# Get top 50 songs for each year
df = pd.DataFrame(columns=['position', 'title', 'artist', 'year'])
for i in range(2018,2023):
    df = df.append(get_top50_from_url(i), ignore_index=True)

df.head()

C:\Users\Surface\AppData\Local\Temp\ipykernel_632\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_632\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_632\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_top50_from_url(i), ignore_index=True)
C:\Users\Surface\AppData\Local\Temp\ipykernel_632\3513559878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(get_to

,position,title,artist,year
0,1,God's Plan,Drake,2018
1,2,I Like It,"Cardi B, Bad Bunny & J Balvin",2018
2,3,In My Feelings,Drake,2018
3,4,Psycho,Post Malone Featuring Ty Dolla $ign,2018
4,5,Nice For What,Drake,2018


In [148]:
# write to csv
df.to_csv('top50.csv', index=False)

# Use lyricsmaster to get lyrics for each song from Genius.com API

In [123]:
def remove_special_chars(file_name):
    # Define special characters to remove
    special_chars = "!@#$%^*()[]{};:/<>?\|`~-=_+’"

    # Remove special characters from file name
    new_file_name = ''.join(c for c in file_name if c not in special_chars)

    return new_file_name

In [136]:
# write a 0 to the index.txt file
with open('index.txt', 'w') as f:
    f.write('0')

def scrape_lyrics():
    # Authenticate with Genius API
    genius = Genius('uWk7jiPtIjLiomyfQS1Y6QvB_Aw3EWQr3ODfulXHBKZU1kBWFBzNwJMQ0rbi3Ig6')
    # create the subdirectory /songs/ if it does not exist
    if not os.path.exists('songs'):
        os.makedirs('songs')
    # read the index.txt file to get the index of the last song that was written to a json file
    with open('index.txt', 'r') as f:
        i = int(f.read())
    # iterate through each row in the dataframe starting at index i
    for row,column in islice(df.iterrows(), i, None):
        song = genius.search_song(column['title'], column['artist'])
        # write the song object to a json file named by the song title and artist name in the folder songs
        file_name = 'song' + str(i) + '.json'
        # modify the file name to remove all special characters that are not allowed in a file name
        file_name = remove_special_chars(file_name)
        song.save_lyrics(file_name, overwrite=True)
        # move the created json into the /songs/ folder
        os.rename(os.getcwd() + '\\' + file_name, os.getcwd() + '\\' + 'songs' + '\\' + file_name)
        # write a txt file with the content i and the name index.txt
        i += 1
        with open('index.txt', 'w') as f:
            f.write(str(i))
        df.loc[row, 'lyrics'] = song.lyrics

while i < len(df):
    try:
        scrape_lyrics()
    except requests.exceptions.Timeout:
        print('Request timed out')
    except requests.exceptions.RequestException as e:
        print('Error:', e)

# delete the index.txt file
os.remove('index.txt')

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

## Read the lyrics from the json files and add them to the dataframe

In [149]:
import json
from glob import glob

In [150]:
# get a list of all the json files in the /songs/ folder
json_files = glob('songs/*.json')

In [154]:
# read each json file and add the lyrics to the dataframe
for file in json_files:
    with open(file, 'r') as f:
        data = json.load(f)
    # extract the i value from the file name
    pattern = r"\d+(?=\.json$)"
    i = int(re.search(pattern, file).group())
    print(i)
    # add the lyrics to the dataframe
    df.loc[i, 'lyrics'] = data['lyrics']

0
1
10
100
101
102
103
104
105
106
107
108
109
11
110
111
112
113
114
115
116
117
118
119
12
120
121
122
123
124
125
126
127
128
129
13
130
131
132
133
134
135
136
137
138
139
14
140
141
142
143
144
145
146
147
148
149
15
150
151
152
153
154
155
156
157
158
159
16
160
161
162
163
164
165
166
167
168
169
17
170
171
172
173
174
175
176
177
178
179
18
180
181
182
183
184
185
186
187
188
189
19
190
191
192
193
194
195
196
197
198
199
2
20
200
201
202
203
204
205
206
207
208
209
21
210
211
212
213
214
215
216
217
218
219
22
220
221
222
223
224
225
226
227
228
229
23
230
231
232
233
234
235
236
237
238
239
24
240
241
242
243
244
245
246
247
248
249
25
26
27
28
29
3
30
31
32
33
34
35
36
37
38
39
4
40
41
42
43
44
45
46
47
48
49
5
50
51
52
53
54
55
56
57
58
59
6
60
61
62
63
64
65
66
67
68
69
7
70
71
72
73
74
75
76
77
78
79
8
80
81
82
83
84
85
86
87
88
89
9
90
91
92
93
94
95
96
97
98
99


In [155]:
# write the dataframe to a csv file
df.to_csv('top50.csv', index=False)